# Bias Mitigation Experiments - German Credit

In [1]:
data_name = 'German_Credit'

In [2]:
# to ignore warnings
import warnings
# warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_eager_execution() # comment out when running TabTransformer models

## [1.](#Table-of-Contents) Prepare Dataset

In [9]:
# read in data as pandas data frame
# as given by german.doc
column_names = ['status', 'month', 'credit_history', 'purpose', 'credit_amount', 'savings', 
                'employment', 'investment_as_income_percentage', 'personal_status', 'other_debtors', 
                'residence_since', 'property', 'age', 'installment_plans', 'housing', 'number_of_credits',
                'skill_level', 'people_liable_for', 'telephone', 'foreign_worker', 'credit']
df = pd.read_csv("../data/German_Credit/german.data", sep=' ', header=None, names=column_names, na_values=[])

# check columns
pd.set_option('display.max_columns', None) # expand all columns
print(df.shape)
df.head()

(1000, 21)


,status,month,credit_history,purpose,credit_amount,savings,employment,investment_as_income_percentage,personal_status,other_debtors,residence_since,property,age,installment_plans,housing,number_of_credits,skill_level,people_liable_for,telephone,foreign_worker,credit
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,4,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,2,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,3,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,4,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,4,A124,53,A143,A153,2,A173,2,A191,A201,2


In [5]:
# data pre-processing
import re
label = 'credit'
protected_attribute = 'sex'
# 1. extract "sex" variable from column "status"
status_map = {'A91': 'male', 'A93': 'male', 'A94': 'male',
              'A92': 'female', 'A95': 'female'}
df['sex'] = df['personal_status'].replace(status_map)

# 2. select features of interest
df = df.drop(columns=['personal_status'], axis=1)

# 3. encode label and potential protected features 
label_mapping = {1: 1, 2: 0} # according to german.doc: 1 = Good, 2 = Bad
sex_mapping = {'male': 1, 'female': 0}

df = df[df.sex.isin(sex_mapping.keys())] # only include samples with certain sex values
df.sex.replace(sex_mapping, inplace=True)
df[label].replace(label_mapping, inplace=True)
df = df.reset_index(drop=True)

print(df.shape)
df.head()

(1000, 21)


,status,month,credit_history,purpose,credit_amount,savings,employment,investment_as_income_percentage,other_debtors,residence_since,property,age,installment_plans,housing,number_of_credits,skill_level,people_liable_for,telephone,foreign_worker,credit,sex
0,A11,6,A34,A43,1169,A65,A75,4,A101,4,A121,67,A143,A152,2,A173,1,A192,A201,1,1
1,A12,48,A32,A43,5951,A61,A73,2,A101,2,A121,22,A143,A152,1,A173,1,A191,A201,0,0
2,A14,12,A34,A46,2096,A61,A74,2,A101,3,A121,49,A143,A152,1,A172,2,A191,A201,1,1
3,A11,42,A32,A42,7882,A61,A74,2,A103,4,A122,45,A143,A153,1,A173,2,A191,A201,1,1
4,A11,24,A33,A40,4870,A61,A73,3,A101,4,A124,53,A143,A153,2,A173,2,A191,A201,0,1


In [8]:
# unique values of the protected feature race
df[protected_attribute].value_counts()

1    690
0    310
Name: sex, dtype: int64

In [7]:
# select descriptive features and target variable
X = df.drop(columns=label, axis=1) # select all features but target feature
y = df[[protected_attribute, label]] # include protected feature in order to AIF360

In [8]:
# set protected attribute as index
X = X.set_index([protected_attribute], append = True, drop = False)
y = y.set_index([protected_attribute], append = True)

# make y data frames to 1d array to pass modeling, but keep index (protected attribute)
y = pd.Series(y[label], index=y.index)

In [9]:
X.to_pickle('../data/{}/{}_X'.format(data_name, data_name))
y.to_pickle('../data/{}/{}_y'.format(data_name, data_name))

## [2.](#Table-of-Contents) Proposed GridSearch Approach 

The GridSearch Approach includes hyperparameter, threshold, and Bias Mitigation

In [10]:
# import FairGridSearch_new as FairGridSearch
from FairGridSearch_combo import *
%load_ext autoreload
%autoreload 2
# allow automatic reloading of changes in FairGridSearch file

C:\Users\Andrew\Documents\study\Third\Ki1\khdv\Proj\Experiment\FairGridSearch_Experiment\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
pip install 'aif360[FairAdapt]'


In [11]:
# None, better
# 'RW', worse
# 'LFR_pre', stuck
# 'LFR_in', stuck
# 'AD', stuck
# 'EGR',better
# 'ROC', better
# 'CEO', better

## 3. Case Study: German Credit

In [12]:
label = 'credit'
protected_attribute = 'sex'

pos_label = 1
priv_group = 1

cv=10
n_jobs=multiprocessing.cpu_count()-1

In [13]:
X = pd.read_pickle('../data/{}/{}_X'.format(data_name, data_name))
y = pd.read_pickle('../data/{}/{}_y'.format(data_name, data_name))

In [14]:
# define desired metric for the use case
ACC_METRIC = 'avg_norm_mcc_score'
FAIR_METRIC = 'abs_avg_spd_score'

### Logistic Regression

In [15]:
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'C':[1, 10],'solver':['liblinear', 'saga'],'penalty':['l2']}, 
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','LFR_pre','LFR_in','AD','EGR','ROC','CEO','RW+ROC','RW+CEO']}

clf_lr = fair_GridsearchCV(base='LR', param_grid=param_grid, 
                           prot_attr=protected_attribute, pos_label=pos_label, priv_group=priv_group,
                           cv=cv, n_jobs=n_jobs)
if __name__ == '__main__':
    clf_lr.fit(X=X, y=y)
    
results_lr = clf_lr.output_table
print(clf_lr._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_lr.to_pickle('./{}_results/{}_results_LR'.format(data_name,data_name))
style_table(results_lr)

  0%|          | 0/4 [00:00<?, ?it/s]

{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 25%|██▌       | 1/4 [00:34<01:44, 34.71s/it]

{'C': 1, 'penalty': 'l2', 'solver': 'saga'}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 50%|█████     | 2/4 [01:11<01:12, 36.06s/it]

...

 75%|███████▌  | 3/4 [01:44<00:34, 34.71s/it]

{'C': 10, 'penalty': 'l2', 'solver': 'saga'}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


100%|██████████| 4/4 [02:21<00:00, 35.33s/it]


base_estimator                                              LR
param              {'C': 1, 'penalty': 'l2', 'solver': 'saga'}
Bias_Mitigation                                             RW
threshold                                                  0.4
Name: 56, dtype: object
Time:  2.3647269416666665 min


,base_estimator,param,Bias_Mitigation,threshold,avg_acc_score,avg_bacc_score,avg_f1_score,avg_auc_score,avg_mcc_score,avg_norm_mcc_score,avg_spd_score,avg_aod_score,avg_eod_score,avg_ford_score,avg_ppvd_score,avg_(1-consistency_score),avg_gei_score,avg_ti_score,cost
0,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",None,0.300000,0.738000,0.591905,0.836347,0.786190,0.289396,0.644698,-0.066994,-0.060125,-0.036950,-0.000079,-0.051646,0.099800,0.076659,0.083690,0.422296
1,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",None,0.400000,0.765000,0.655952,0.846905,0.786190,0.386606,0.693303,-0.088203,-0.077101,-0.038273,-0.079435,-0.030706,0.152600,0.083957,0.099237,0.394900
2,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",None,0.500000,0.760000,0.687619,0.834817,0.786190,0.404027,0.702014,-0.139313,-0.133722,-0.073295,-0.068408,-0.001388,0.205800,0.106288,0.140002,0.437299
3,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",None,0.600000,0.744000,0.706667,0.813147,0.786190,0.408403,0.704201,-0.116301,-0.098689,-0.062672,-0.068371,-0.006087,0.244000,0.134859,0.189776,0.412100
4,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",None,0.700000,0.719000,0.715476,0.781941,0.786190,0.404405,0.702203,-0.115555,-0.104395,-0.051571,-0.096930,0.025323,0.257400,0.169881,0.248275,0.413352
5,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",RW,0.300000,0.740000,0.591429,0.838353,0.783619,0.289342,0.644671,-0.010974,0.010906,-0.010859,-0.001032,-0.076735,0.092200,0.074164,0.079498,0.366303
6,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",RW,0.400000,0.752000,0.640000,0.838584,0.783619,0.346948,0.673474,-0.009007,0.024947,-0.005364,-0.076094,-0.077583,0.150400,0.088681,0.106650,0.335533
7,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",RW,0.500000,0.757000,0.680714,0.833535,0.783619,0.393659,0.696830,-0.044898,-0.030746,-0.000802,-0.139112,-0.036104,0.201800,0.106068,0.138751,0.348068
8,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",RW,0.600000,0.749000,0.711190,0.816945,0.783619,0.419296,0.709648,-0.045681,-0.019873,-0.006461,-0.098618,-0.039917,0.240400,0.131720,0.184984,0.336034
9,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",RW,0.700000,0.715000,0.710714,0.779159,0.783619,0.394761,0.697380,-0.057764,-0.043136,0.000794,-0.120115,-0.003542,0.259400,0.172145,0.251144,0.360384


### Random Forest

In [17]:
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'n_estimators':[10, 50],'criterion':['gini', 'entropy'], 'max_depth':[16]}, 
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','LFR_pre','EGR','ROC','CEO','RW+ROC','RW+CEO']}

clf_rf = fair_GridsearchCV(base='RF', param_grid=param_grid,
                           prot_attr=protected_attribute, pos_label=pos_label, priv_group=priv_group,
                           cv=cv, n_jobs=n_jobs)
if __name__ == '__main__':
    clf_rf.fit(X=X, y=y)
    
results_rf = clf_rf.output_table
print(clf_rf._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_rf.to_pickle('./{}_results/{}_results_RF'.format(data_name, data_name))
style_table(results_rf)

  0%|          | 0/4 [00:00<?, ?it/s]

{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 25%|██▌       | 1/4 [00:35<01:45, 35.01s/it]

{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 50}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 50%|█████     | 2/4 [02:28<02:42, 81.05s/it]

...

 75%|███████▌  | 3/4 [03:05<01:01, 61.14s/it]

{'criterion': 'entropy', 'max_depth': 16, 'n_estimators': 50}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


100%|██████████| 4/4 [04:24<00:00, 66.18s/it]


base_estimator                                                    RF
param              {'criterion': 'gini', 'max_depth': 16, 'n_esti...
Bias_Mitigation                                                  EGR
threshold                                                        0.3
Name: 55, dtype: object
Time:  4.422317533333333 min


,base_estimator,param,Bias_Mitigation,threshold,avg_acc_score,avg_bacc_score,avg_f1_score,avg_auc_score,avg_mcc_score,avg_norm_mcc_score,avg_spd_score,avg_aod_score,avg_eod_score,avg_ford_score,avg_ppvd_score,avg_(1-consistency_score),avg_gei_score,avg_ti_score,cost
0,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",None,0.300000,0.710000,0.529048,0.825718,0.744643,0.144733,0.572367,-0.003389,0.009877,-0.007681,0.065000,-0.077256,0.055000,0.068947,0.069521,0.431022
1,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",None,0.400000,0.725000,0.579762,0.827487,0.744643,0.241221,0.620611,-0.046625,-0.042719,-0.012707,-0.122381,-0.048236,0.132000,0.083083,0.094573,0.426015
2,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",None,0.500000,0.741000,0.638810,0.828586,0.744643,0.324428,0.662214,-0.055278,-0.031037,-0.033636,0.003175,-0.052671,0.204200,0.099822,0.125527,0.393064
3,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",None,0.600000,0.731000,0.672619,0.809599,0.744643,0.351776,0.675888,-0.096678,-0.063897,-0.081400,-0.000180,-0.049456,0.262000,0.130632,0.179266,0.420790
4,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",None,0.700000,0.682000,0.674762,0.752004,0.744643,0.327267,0.663633,-0.140192,-0.113609,-0.115337,-0.031962,-0.022539,0.301200,0.195367,0.280767,0.476559
5,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",RW,0.300000,0.710000,0.530952,0.825316,0.740190,0.140178,0.570089,0.004101,0.014595,0.002111,-0.060000,-0.075453,0.053200,0.070020,0.071433,0.434013
6,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",RW,0.400000,0.719000,0.568810,0.824689,0.740190,0.212862,0.606431,-0.033322,-0.022380,-0.012316,0.048333,-0.057329,0.117000,0.082853,0.094063,0.426891
7,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",RW,0.500000,0.748000,0.645714,0.833402,0.740190,0.345437,0.672719,-0.039461,-0.008652,-0.029326,0.012816,-0.067097,0.196800,0.096451,0.119918,0.366743
8,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",RW,0.600000,0.718000,0.664286,0.797834,0.740190,0.331200,0.665600,-0.038696,-0.010100,-0.025568,-0.058177,-0.063259,0.267600,0.141742,0.196114,0.373096
9,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",RW,0.700000,0.682000,0.678571,0.750231,0.740190,0.334217,0.667108,-0.077994,-0.048752,-0.060374,-0.059207,-0.047884,0.298400,0.197065,0.284779,0.410886


### Gradient Boosting

In [15]:
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'n_estimators':[10, 50],'criterion':['friedman_mse'],'max_depth':[8, 32]}, 
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','LFR_pre','EGR','ROC','CEO','RW+ROC','RW+CEO']}

clf_gb = fair_GridsearchCV(base='GB', param_grid=param_grid,
                           prot_attr=protected_attribute, pos_label=pos_label, priv_group=priv_group,
                           cv=cv, n_jobs=n_jobs)
if __name__ == '__main__':
    clf_gb.fit(X=X, y=y)
    
results_gb = clf_gb.output_table
print(clf_gb._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_gb.to_pickle('./{}_results/{}_results_GB'.format(data_name, data_name))
# style_table(results_gb)

  0%|          | 0/4 [00:00<?, ?it/s]

{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 25%|██▌       | 1/4 [00:36<01:50, 36.74s/it]

{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 50}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 50%|█████     | 2/4 [05:54<06:43, 201.83s/it]

{'criterion': 'friedman_mse', 'max_depth': 32, 'n_estimators': 10}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 75%|███████▌  | 3/4 [08:10<02:51, 171.74s/it]

{'criterion': 'friedman_mse', 'max_depth': 32, 'n_estimators': 50}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


100%|██████████| 4/4 [14:39<00:00, 219.86s/it]


base_estimator                                                    GB
param              {'criterion': 'friedman_mse', 'max_depth': 8, ...
Bias_Mitigation                                                  ROC
threshold                                                        0.5
Name: 62, dtype: object
Time:  14.667178363333333 min


In [10]:
style_table(results_gb)

,base_estimator,param,Bias_Mitigation,threshold,avg_acc_score,avg_bacc_score,avg_f1_score,avg_auc_score,avg_mcc_score,avg_norm_mcc_score,avg_spd_score,avg_aod_score,avg_eod_score,avg_ford_score,avg_ppvd_score,avg_(1-consistency_score),avg_gei_score,avg_ti_score,cost
0,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",None,0.300000,0.712000,0.534286,0.826399,0.733405,0.134516,0.567258,0.009109,0.014645,0.010157,-0.130000,-0.073064,0.048800,0.069812,0.071190,0.441851
1,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",None,0.400000,0.733000,0.584524,0.833677,0.733405,0.264133,0.632066,-0.002496,0.008424,0.007471,-0.130029,-0.069423,0.112600,0.077382,0.084965,0.370430
2,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",None,0.500000,0.737000,0.628333,0.827152,0.733405,0.308601,0.654301,-0.000114,0.017349,0.013606,-0.129975,-0.067648,0.194200,0.098378,0.122361,0.345813
3,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",None,0.600000,0.715000,0.647857,0.800277,0.733405,0.303870,0.651935,-0.040123,-0.011933,-0.033677,-0.039669,-0.068946,0.267000,0.134481,0.183795,0.388188
4,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",None,0.700000,0.683000,0.673571,0.754977,0.733405,0.323648,0.661824,-0.086862,-0.049035,-0.085928,-0.021232,-0.063871,0.326000,0.190555,0.275641,0.425038
5,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",RW,0.300000,0.707000,0.529762,0.822999,0.732500,0.110861,0.555430,0.004852,0.008121,0.003444,-0.175000,-0.074052,0.052800,0.072313,0.075479,0.449422
6,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",RW,0.400000,0.725000,0.579762,0.827641,0.732500,0.233678,0.616839,-0.017719,-0.025552,0.017871,-0.210931,-0.047122,0.124200,0.082836,0.094372,0.400880
7,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",RW,0.500000,0.717000,0.609286,0.813116,0.732500,0.253575,0.626787,-0.000581,0.005830,0.021791,-0.147544,-0.058406,0.201000,0.108698,0.139294,0.373793
8,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",RW,0.600000,0.722000,0.663333,0.803246,0.732500,0.328955,0.664478,-0.017873,0.011470,-0.012745,-0.048876,-0.071927,0.263800,0.134973,0.186305,0.353395
9,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",RW,0.700000,0.682000,0.671905,0.754084,0.732500,0.321163,0.660581,-0.076326,-0.050347,-0.055222,-0.060412,-0.044318,0.315800,0.191140,0.276123,0.415745


### Support Vector Machine

In [19]:
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'kernel':['rbf','linear','poly','sigmoid'],'gamma':['scale']},
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','LFR_pre','EGR','ROC','CEO','RW+ROC','RW+CEO']}

clf_svm = fair_GridsearchCV(base='SVM', param_grid=param_grid,
                            prot_attr=protected_attribute, pos_label=pos_label, priv_group=priv_group,
                            cv=cv, n_jobs=n_jobs)
if __name__ == '__main__':
    clf_svm.fit(X=X, y=y)
    
results_svm = clf_svm.output_table
print(clf_svm._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_svm.to_pickle('./{}_results/{}_results_SVM'.format(data_name, data_name))
style_table(results_svm)

  0%|          | 0/4 [00:00<?, ?it/s]

{'gamma': 'scale', 'kernel': 'rbf'}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 25%|██▌       | 1/4 [01:14<03:43, 74.64s/it]

{'gamma': 'scale', 'kernel': 'linear'}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 50%|█████     | 2/4 [02:37<02:39, 79.71s/it]

...

 75%|███████▌  | 3/4 [03:38<01:10, 70.98s/it]

{'gamma': 'scale', 'kernel': 'sigmoid'}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


100%|██████████| 4/4 [04:29<00:00, 67.48s/it]


base_estimator                                         SVM
param              {'gamma': 'scale', 'kernel': 'sigmoid'}
Bias_Mitigation                                         RW
threshold                                              0.7
Name: 129, dtype: object
Time:  4.507528546666666 min


,base_estimator,param,Bias_Mitigation,threshold,avg_acc_score,avg_bacc_score,avg_f1_score,avg_auc_score,avg_mcc_score,avg_norm_mcc_score,avg_spd_score,avg_aod_score,avg_eod_score,avg_ford_score,avg_ppvd_score,avg_(1-consistency_score),avg_gei_score,avg_ti_score,cost
0,SVM,"{'gamma': 'scale', 'kernel': 'rbf'}",None,0.300000,0.749000,0.595952,0.845235,0.793690,0.324107,0.662053,-0.066609,-0.063815,-0.034539,0.033571,-0.046903,0.095200,0.067663,0.068170,0.404556
1,SVM,"{'gamma': 'scale', 'kernel': 'rbf'}",None,0.400000,0.763000,0.636429,0.849236,0.793690,0.372009,0.686004,-0.108870,-0.116295,-0.045888,-0.010635,-0.018646,0.139400,0.075551,0.083500,0.422865
2,SVM,"{'gamma': 'scale', 'kernel': 'rbf'}",None,0.500000,0.764000,0.671429,0.842458,0.793690,0.395607,0.697804,-0.098229,-0.100238,-0.024016,-0.110274,-0.005620,0.190000,0.093654,0.116640,0.400425
3,SVM,"{'gamma': 'scale', 'kernel': 'rbf'}",None,0.600000,0.748000,0.695238,0.820654,0.793690,0.397523,0.698762,-0.116054,-0.108964,-0.059548,-0.064829,-0.005110,0.231400,0.124209,0.170473,0.417293
4,SVM,"{'gamma': 'scale', 'kernel': 'rbf'}",None,0.700000,0.725000,0.717857,0.788756,0.793690,0.409954,0.704977,-0.132393,-0.118146,-0.077216,-0.069331,0.015392,0.267400,0.162666,0.238034,0.427415
5,SVM,"{'gamma': 'scale', 'kernel': 'rbf'}",RW,0.300000,0.745000,0.590238,0.842973,0.794452,0.308345,0.654173,-0.050110,-0.039339,-0.032539,0.030476,-0.059552,0.092400,0.068464,0.069481,0.395937
6,SVM,"{'gamma': 'scale', 'kernel': 'rbf'}",RW,0.400000,0.755000,0.625952,0.844303,0.794452,0.345636,0.672818,-0.064409,-0.057806,-0.026652,-0.046205,-0.043054,0.138600,0.078025,0.087346,0.391591
7,SVM,"{'gamma': 'scale', 'kernel': 'rbf'}",RW,0.500000,0.771000,0.677381,0.847699,0.794452,0.413465,0.706733,-0.082798,-0.080469,-0.015818,-0.119129,-0.014715,0.185200,0.089337,0.109772,0.376065
8,SVM,"{'gamma': 'scale', 'kernel': 'rbf'}",RW,0.600000,0.747000,0.693571,0.820068,0.794452,0.394536,0.697268,-0.069032,-0.064408,-0.008223,-0.119507,-0.011500,0.229800,0.124492,0.170701,0.371764
9,SVM,"{'gamma': 'scale', 'kernel': 'rbf'}",RW,0.700000,0.722000,0.716667,0.785517,0.794452,0.407061,0.703531,-0.077435,-0.058079,-0.034368,-0.085243,-0.014875,0.273200,0.165985,0.242970,0.373904


### Gaussian Naive Bayes

In [20]:
# model for Reweighing
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'var_smoothing': np.logspace(0,-9, num=4)}, 
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','LFR_pre','EGR','ROC','CEO','RW+ROC','RW+CEO']}

clf_nb = fair_GridsearchCV(base='NB', param_grid=param_grid, 
                           prot_attr=protected_attribute, pos_label=pos_label, priv_group=priv_group,
                           cv=cv, n_jobs=n_jobs)
if __name__ == '__main__':
    clf_nb.fit(X=X, y=y)
    
results_nb = clf_nb.output_table
# print(clf._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_nb.to_pickle('./{}_results/{}_results_NB'.format(data_name, data_name))
style_table(results_nb)

  0%|          | 0/4 [00:00<?, ?it/s]

{'var_smoothing': 1.0}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 25%|██▌       | 1/4 [00:34<01:43, 34.54s/it]

{'var_smoothing': 0.001}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 50%|█████     | 2/4 [01:11<01:12, 36.16s/it]

...

 75%|███████▌  | 3/4 [01:42<00:33, 33.82s/it]

{'var_smoothing': 1e-09}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


100%|██████████| 4/4 [02:11<00:00, 32.77s/it]


Time:  2.1909912383333374 min


,base_estimator,param,Bias_Mitigation,threshold,avg_acc_score,avg_bacc_score,avg_f1_score,avg_auc_score,avg_mcc_score,avg_norm_mcc_score,avg_spd_score,avg_aod_score,avg_eod_score,avg_ford_score,avg_ppvd_score,avg_(1-consistency_score),avg_gei_score,avg_ti_score,cost
0,NB,{'var_smoothing': 1.0},None,0.300000,0.720000,0.615238,0.814238,0.762190,0.267205,0.633603,-0.008230,0.010439,0.004797,-0.114481,-0.067995,0.134400,0.109215,0.140160,0.374627
1,NB,{'var_smoothing': 1.0},None,0.400000,0.723000,0.638333,0.811114,0.762190,0.299185,0.649593,-0.040703,-0.028465,-0.011298,-0.100546,-0.048514,0.166600,0.119499,0.158546,0.391110
2,NB,{'var_smoothing': 1.0},None,0.500000,0.733000,0.675952,0.810798,0.762190,0.357722,0.678861,-0.085197,-0.082568,-0.025281,-0.124982,-0.007776,0.195000,0.129800,0.178671,0.406336
3,NB,{'var_smoothing': 1.0},None,0.600000,0.723000,0.691667,0.795294,0.762190,0.370574,0.685287,-0.077216,-0.056453,-0.042345,-0.075534,-0.027438,0.214000,0.150219,0.214295,0.391929
4,NB,{'var_smoothing': 1.0},None,0.700000,0.715000,0.709762,0.779660,0.762190,0.393646,0.696823,-0.060190,-0.030549,-0.025901,-0.079599,-0.028906,0.232400,0.170612,0.249635,0.363367
5,NB,{'var_smoothing': 1.0},RW,0.300000,0.722000,0.618571,0.815396,0.758143,0.273073,0.636537,0.014980,0.035595,0.025876,-0.137603,-0.071047,0.131000,0.108849,0.139806,0.378444
6,NB,{'var_smoothing': 1.0},RW,0.400000,0.720000,0.630476,0.810101,0.758143,0.287119,0.643559,0.002055,0.027755,0.010278,-0.108648,-0.072864,0.152800,0.118486,0.156183,0.358496
7,NB,{'var_smoothing': 1.0},RW,0.500000,0.723000,0.663095,0.804151,0.758143,0.331318,0.665659,-0.033196,-0.017875,-0.003148,-0.097962,-0.044732,0.187200,0.133898,0.184344,0.367536
8,NB,{'var_smoothing': 1.0},RW,0.600000,0.719000,0.682143,0.793622,0.758143,0.355565,0.677782,-0.050112,-0.034009,-0.015017,-0.104020,-0.035769,0.207800,0.150117,0.212487,0.372329
9,NB,{'var_smoothing': 1.0},RW,0.700000,0.720000,0.712381,0.784704,0.758143,0.399999,0.699999,-0.042195,-0.013315,-0.010270,-0.093204,-0.036934,0.233200,0.165457,0.242159,0.342195


### TabTransformer

In [14]:
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'epochs':[20, 30],'learing_rate':[1e-04, 1e-05]},
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','ROC','CEO','RW+ROC','RW+CEO']}

clf_tab = fair_GridsearchCV(base='TabTrans',param_grid=param_grid, 
                            prot_attr=protected_attribute, pos_label=pos_label, priv_group=priv_group,
                            cv=cv, n_jobs=n_jobs)

clf_tab.fit(X=X, y=y)
results_tab = clf_tab.output_table
# print(clf._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_tab.to_pickle('./{}_results/{}_results_TabTrans'.format(data_name, data_name))

style_table(results_tab)

  0%|          | 0/4 [00:00<?, ?it/s]

{'epochs': 20, 'learing_rate': 0.0001}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 25%|██▌       | 1/4 [13:04<39:14, 784.69s/it]

{'epochs': 20, 'learing_rate': 1e-05}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 50%|█████     | 2/4 [23:57<23:34, 707.00s/it]

...

 75%|███████▌  | 3/4 [34:34<11:15, 675.15s/it]

{'epochs': 30, 'learing_rate': 1e-05}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


100%|██████████| 4/4 [48:43<00:00, 730.95s/it]


Time:  48.74462431 min


,base_estimator,param,Bias_Mitigation,threshold,avg_acc_score,avg_bacc_score,avg_f1_score,avg_auc_score,avg_mcc_score,avg_norm_mcc_score,avg_spd_score,avg_aod_score,avg_eod_score,avg_ford_score,avg_ppvd_score,avg_(1-consistency_score),avg_gei_score,avg_ti_score,cost
0,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",None,0.300000,0.681000,0.623571,0.758157,0.734333,0.275225,0.637612,-0.074247,-0.048817,-0.080496,-0.027464,-0.063478,0.256800,0.171318,0.234124,0.436635
1,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",None,0.400000,0.646000,0.645238,0.694237,0.734333,0.299808,0.649904,-0.058373,-0.022509,-0.078585,-0.019977,-0.089486,0.307200,0.246231,0.341025,0.408469
2,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",None,0.500000,0.594000,0.640476,0.608378,0.734333,0.280603,0.640302,-0.056428,-0.024893,-0.054380,-0.077615,-0.196904,0.331200,0.355555,0.467981,0.416126
3,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",None,0.600000,0.539000,0.620238,0.516032,0.734333,0.237765,0.618882,-0.026178,-0.005371,-0.012598,-0.081555,-0.052500,0.311600,0.469540,0.590188,0.407295
4,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",None,0.700000,0.496000,0.619048,0.428911,0.734333,0.248298,0.624149,-0.057788,-0.039733,-0.039562,-0.077422,-0.169415,0.257400,0.568538,0.706329,0.433639
5,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",RW,0.300000,0.708000,0.636190,0.792170,0.756333,0.300920,0.650460,-0.066096,-0.042343,-0.048153,-0.034991,-0.055676,0.257200,0.137885,0.187665,0.415636
6,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",RW,0.400000,0.697000,0.669286,0.766751,0.756333,0.340822,0.670411,-0.102429,-0.080484,-0.077016,-0.068148,-0.028373,0.322200,0.174246,0.247075,0.432019
7,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",RW,0.500000,0.648000,0.674286,0.695358,0.756333,0.334048,0.667024,-0.057311,-0.032721,-0.030198,-0.093427,-0.024179,0.370200,0.254856,0.362437,0.390287
8,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",RW,0.600000,0.609000,0.670238,0.632884,0.756333,0.329355,0.664678,-0.051490,-0.021219,-0.029023,-0.085101,-0.042959,0.358800,0.321719,0.451774,0.386812
9,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",RW,0.700000,0.544000,0.641905,0.522688,0.756333,0.286926,0.643463,-0.063282,-0.042236,-0.037307,-0.081202,-0.091551,0.311200,0.450451,0.592167,0.419819
